<a href="https://colab.research.google.com/github/VasilinaFrolova/MyPain/blob/main/%D0%A4%D1%80%D0%BE%D0%BB%D0%BE%D0%B2%D0%B0_%D0%92%D0%B0%D1%81%D0%B8%D0%BB%D0%B8%D0%BD%D0%B0_%22w2v_hw_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [1]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 27.1 MB/s eta 0:00:00


## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [2]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 128.1/128.1MB downloaded
Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Базовые операции с векторами**

In [3]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [4]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Слова, похожие на 'python':
  monty: 0.6886
  php: 0.5865
  perl: 0.5784
  cleese: 0.5447
  flipper: 0.5113


*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [5]:
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-100")

2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [8]:
def find_similar_words(word, top_n=10):
  try:
        # Находим наиболее близкие слова
        similar_words = model.most_similar(word, topn=top_n)
        return similar_words
  except KeyError:
        print(f"Слово '{word}' отсутствует в словаре модели")
        return []
  except Exception as e:
        print(f"Произошла ошибка: {e}")
        return []

3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [10]:
from gensim.models import Word2Vec

# Данные для обучения
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

# Обучение модели Word2Vec с указанными параметрами
model = Word2Vec(
    sentences=cooking_sentences,
    vector_size=50,          # размер вектора
    window=3,                # размер окна
    min_count=1,             # минимальная частота слова
    workers=2,               # количество потоков
    sg=1                     # использование skip-gram (1 = skip-gram, 0 = CBOW)
)

# Функция для поиска похожих слов
def find_similar_words(word, top_n=10):
    """
    Находит наиболее близкие слова к заданному слову
    """
    try:
        similar_words = model.wv.most_similar(word, topn=top_n)

        print(f"\n{top_n} наиболее близких слов к '{word}':")
        print("-" * 40)
        for i, (similar_word, similarity) in enumerate(similar_words, 1):
            print(f"{i:2d}. {similar_word:<15} {similarity:.4f}")

        return similar_words

    except KeyError:
        print(f"Слово '{word}' отсутствует в словаре модели")
        return []
    except Exception as e:
        print(f"Произошла ошибка: {e}")
        return []

# Проверка модели
print("Информация о модели:")
print(f"Размер словаря: {len(model.wv.key_to_index)}")
print(f"Размер вектора: {model.wv.vector_size}")
print(f"Архитектура: {'Skip-gram' if model.sg == 1 else 'CBOW'}")

# Тестирование на нескольких словах
test_words = ['варить', 'овощи', 'тесто', 'духовка']
for word in test_words:
    find_similar_words(word, top_n=5)

# Дополнительная информация о модели
print(f"\nСлова в словаре: {list(model.wv.key_to_index.keys())}")

Информация о модели:
Размер словаря: 65
Размер вектора: 50
Архитектура: Skip-gram

5 наиболее близких слов к 'варить':
----------------------------------------
 1. вино            0.2398
 2. ингредиенты     0.2172
 3. хлеб            0.1938
 4. брокколи        0.1846
 5. кипятить        0.1711

5 наиболее близких слов к 'овощи':
----------------------------------------
 1. мариновать      0.2716
 2. хлеб            0.2691
 3. гриль           0.2546
 4. фольга          0.2409
 5. сахар           0.2108

5 наиболее близких слов к 'тесто':
----------------------------------------
 1. жарить          0.2416
 2. масло           0.2372
 3. помидоры        0.2005
 4. тушить          0.1941
 5. огурцы          0.1690

5 наиболее близких слов к 'духовка':
----------------------------------------
 1. ингредиенты     0.3199
 2. десерт          0.3064
 3. холодильник     0.2705
 4. питание         0.2243
 5. пирог           0.2142

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духов

In [11]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [12]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [13]:
# Найдите слова, похожие на "духовка"
try:
    similar_oven = model.wv.most_similar('духовка', topn=5)
    print("\nСлова, похожие на 'духовка':")
    for word, score in similar_oven:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")
# Найдите слова, похожие на "овощи"
try:
    similar_veggies = model.wv.most_similar('овощи', topn=5)
    print("\nСлова, похожие на 'овощи':")
    for word, score in similar_veggies:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")


Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142

Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [15]:
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [16]:
# Находим слова, похожие на "духовка"
try:
    similar = ft_model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

# Находим слова, похожие на "духовка"
try:
    similar_oven = ft_model.wv.most_similar('духовка', topn=5)
    print("\nСлова, похожие на 'духовка':")
    for word, score in similar_oven:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

# Находим слова, похожие на "овощи"
try:
    similar_veggies = ft_model.wv.most_similar('овощи', topn=5)
    print("\nСлова, похожие на 'овощи':")
    for word, score in similar_veggies:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'варить':
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622

Слова, похожие на 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944

Слова, похожие на 'овощи':
  жарить: 0.2960
  фольга: 0.2574
  морковь: 0.2297
  соус: 0.2172
  торт: 0.2094


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [19]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Тестируем 3 слова с опечатками на кулинарную тематику
test_words_with_typos = ['ворить', 'духовк', 'овоки']

for word in test_words_with_typos:
    compare_models(word)


Сравнение для слова: 'ворить'
  Word2Vec: слово не найдено
  FastText: ['жарить', 'ингредиенты']

Сравнение для слова: 'духовк'
  Word2Vec: слово не найдено
  FastText: ['духовка', 'вино']

Сравнение для слова: 'овоки'
  Word2Vec: слово не найдено
  FastText: ['барбекю', 'яичница']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [20]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [21]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [22]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [23]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [25]:
# Сравниваем схожесть doc_2 и doc_4
compare_documents(2, 4)

Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [26]:
# Находим самый похожий документ на doc_1
print(f"\nСамый похожий документ на doc_1:")
similar_to_doc1 = doc_model.dv.most_similar("doc_1", topn=1)
for doc_tag, similarity in similar_to_doc1:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")


Самый похожий документ на doc_1:
  doc_0: 0.2735
    Текст: machine learning is interesting


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [27]:
from gensim.models import Word2Vec

# Кулинарный датасет
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

# Обучаем модели с разной размерностью
vector_sizes = [10, 50, 100]
models = {}

for size in vector_sizes:
    print(f"\nОбучение модели с размерностью {size}...")
    models[size] = Word2Vec(
        sentences=cooking_sentences,
        vector_size=size,
        window=3,
        min_count=1,
        workers=2,
        sg=1
    )
    print(f"Модель с размерностью {size} обучена!")
    print(f"Размер словаря: {len(models[size].wv.key_to_index)}")

# Функция для сравнения качества моделей
def compare_models_quality(test_words):
    """Сравнивает качество моделей с разной размерностью"""
    print("\n" + "="*60)
    print("СРАВНЕНИЕ КАЧЕСТВА МОДЕЛЕЙ С РАЗНОЙ РАЗМЕРНОСТЬЮ")
    print("="*60)

    for word in test_words:
        print(f"\nСлово: '{word}'")
        print("-" * 40)

        for size in vector_sizes:
            model = models[size]
            try:
                similar_words = model.wv.most_similar(word, topn=3)
                print(f"  Размерность {size:3d}: {[w for w, _ in similar_words]}")
            except KeyError:
                print(f"  Размерность {size:3d}: слово не найдено")

# Тестируем на 3 кулинарных словах
test_words = ['варить', 'овощи', 'тесто']
compare_models_quality(test_words)

# Дополнительная проверка с коэффициентами схожести
print("\n" + "="*60)
print("ПОДРОБНОЕ СРАВНЕНИЕ С КОЭФФИЦИЕНТАМИ СХОЖЕСТИ")
print("="*60)

for word in test_words:
    print(f"\nСлово: '{word}'")
    print("-" * 40)

    for size in vector_sizes:
        model = models[size]
        try:
            similar_words = model.wv.most_similar(word, topn=2)
            print(f"  Размерность {size:3d}:")
            for similar_word, similarity in similar_words:
                print(f"    {similar_word}: {similarity:.4f}")
        except KeyError:
            print(f"  Размерность {size:3d}: слово не найдено")


Обучение модели с размерностью 10...
Модель с размерностью 10 обучена!
Размер словаря: 65

Обучение модели с размерностью 50...
Модель с размерностью 50 обучена!
Размер словаря: 65

Обучение модели с размерностью 100...
Модель с размерностью 100 обучена!
Размер словаря: 65

СРАВНЕНИЕ КАЧЕСТВА МОДЕЛЕЙ С РАЗНОЙ РАЗМЕРНОСТЬЮ

Слово: 'варить'
----------------------------------------
  Размерность  10: ['рыба', 'сковорода', 'хлеб']
  Размерность  50: ['вино', 'ингредиенты', 'хлеб']
  Размерность 100: ['чашка', 'вино', 'сковорода']

Слово: 'овощи'
----------------------------------------
  Размерность  10: ['жарить', 'фольга', 'горшок']
  Размерность  50: ['мариновать', 'хлеб', 'гриль']
  Размерность 100: ['взбивать', 'питание', 'бекон']

Слово: 'тесто'
----------------------------------------
  Размерность  10: ['специи', 'тушить', 'барбекю']
  Размерность  50: ['жарить', 'масло', 'помидоры']
  Размерность 100: ['десерт', 'месить', 'горшок']

ПОДРОБНОЕ СРАВНЕНИЕ С КОЭФФИЦИЕНТАМИ СХОЖЕСТИ

